In [71]:
import pandas as pd
import requests
import json
from time import sleep
import concurrent.futures
import time

In [80]:
headers={
'user-agent':'Mozilla/5.0 (iPad; CPU OS 13_3 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) CriOS/87.0.4280.77 Mobile/15E148 Safari/604.1',
'accept': 'application/json, text/plain, */*',
'accept-encoding': 'gzip, deflate, br',
'accept-language': 'en-US,en;q=0.9,vi;q=0.8',
}

params={
    'limit': '40',
    'include': 'advertisement',
    'is_mweb': '1',
    'aggregations': '2',
    'q': 'điện thoại samsung',
}

In [88]:
product_list=[]
urls=[]
temp_url=f'https://shopee.vn/search?category=11036030&keyword=%C4%91i%E1%BB%87n%20tho%E1%BA%A1i%20di%20%C4%91%E1%BB%99ng&page='
for i in range(0,4):
    url=temp_url+str(i)
    urls.append(url)
    
def Get_List_ID(url):
    response=requests.get(url,headers=headers,params=params)
    if response.status_code == 200:
        print(f'-------Crawl page {url[-1]} successfully!!!--------')
    for record in response.json().get('data'):
        product_list.append(record.get('id'))
    

In [89]:
with concurrent.futures.ThreadPoolExecutor() as executor:
    executor.map(Get_List_ID,urls)

-------Crawl page successfully!!!--------
-------Crawl page successfully!!!--------
-------Crawl page successfully!!!--------
-------Crawl page successfully!!!--------
-------Crawl page successfully!!!--------
-------Crawl page successfully!!!--------
-------Crawl page successfully!!!--------
-------Crawl page successfully!!!--------
-------Crawl page successfully!!!--------
-------Crawl page successfully!!!--------
-------Crawl page successfully!!!--------
-------Crawl page successfully!!!--------
-------Crawl page successfully!!!--------
-------Crawl page successfully!!!--------
-------Crawl page successfully!!!--------
-------Crawl page successfully!!!--------
-------Crawl page successfully!!!--------
-------Crawl page successfully!!!--------
-------Crawl page successfully!!!--------
-------Crawl page successfully!!!--------
-------Crawl page successfully!!!--------
-------Crawl page successfully!!!--------
-------Crawl page successfully!!!--------
-------Crawl page successfully!!!-

In [90]:
product_list

[215746882,
 188837747,
 200995800,
 230018349,
 172319352,
 217560479,
 120782084,
 247016383,
 188043025,
 199902129,
 172313857,
 212329857,
 189677315,
 174630782,
 210280612,
 188833574,
 140144656,
 176438844,
 192859201,
 159110705,
 171592667,
 166613831,
 188868071,
 172175893,
 198254452,
 161158099,
 199196755,
 191689944,
 126703130,
 168317790,
 204802771,
 162473705,
 188835531,
 201490941,
 168320000,
 191916060,
 247299188,
 195330986,
 11303206,
 230157205,
 208193462,
 144142520,
 169520871,
 159887168,
 181699278,
 187266104,
 188833794,
 193207892,
 169547788,
 195334253,
 189937468,
 58065346,
 193310183,
 225910655,
 168183442,
 177343384,
 207883258,
 194041787,
 194041977,
 163039754,
 193208070,
 187955862,
 213587929,
 188832832,
 181517965,
 175193503,
 244182370,
 193206340,
 217559440,
 168304456,
 114092342,
 212324916,
 197605015,
 199026890,
 191581836,
 174589577,
 181264344,
 217354004,
 198732327,
 225235573,
 196348143,
 225271277,
 225228869,
 18993

In [40]:
masterProduct=[]
productDetail=[]
Marketing=[]
def Get_Product_Information(id):
    url=f'https://tiki.vn/api/v2/products/{id}'
    print(url)
    response=requests.get(url,headers=headers,params=params)
    text=response.content
    data=json.loads(text)

    master_=[]
    master_.append(data['id'])
    master_.append(data['name'])
    master_.append(data['categories']['name'])
    master_.append(data['brand']['name'])
    master_.append(data['current_seller']['id'])
    master_.append(data['current_seller']['name'])
    masterProduct.append(master_)
    
    for i in range(0,len(data['configurable_products'])):
        detail_=[]
        detail_.append(data['id'])
        detail_.append(int(data['configurable_products'][i]['sku']))
        detail_.append(data['configurable_products'][i]['option1'])
        detail_.append(data['configurable_products'][i]['price'])
        productDetail.append(detail_)
        
    marketing_=[]
    marketing_.append(data['id'])
    marketing_.append(data['review_count'])
    marketing_.append(data['rating_average'])
    marketing_.append(data['all_time_quantity_sold'])
    Marketing.append(marketing_)

In [ ]:
with concurrent.futures.ThreadPoolExecutor() as executor:
    executor.map(Get_Product_Information,product_list)
end=time.perf_counter()

In [42]:
df_MasterProduct=pd.DataFrame(data=masterProduct,columns=['ID','Name','Category','Brand','Shop ID','Shop name'])
df_ProductDetail=pd.DataFrame(data=productDetail,columns=['Product ID','SKU','Option','Price'])
df_Marketing=pd.DataFrame(data=Marketing,columns=['Product ID','Count review','Average score','Sold items'])

In [43]:
df_MasterProduct

,ID,Name,Category,Brand,Shop ID,Shop name
0,170497385,Điện thoại Samsung Galaxy A53 5G (8GB/128GB) -...,Điện thoại Smartphone,Samsung,1,Tiki Trading
1,162593187,Điện thoại Samsung Galaxy S22 Ultra 5G (12GB/5...,Điện thoại Smartphone,Samsung,1,Tiki Trading
2,192216530,Điện Thoại Samsung Galaxy A13 (4GB/128GB) - Hà...,Điện thoại Smartphone,Samsung,10016,Antam Trading
3,197966909,Điện Thoại Samsung A04s 4GB/64GB - Hàng Chính ...,Điện thoại Smartphone,Samsung,29952,Hồng Hạnh Mobile
4,162593136,Điện thoại Samsung Galaxy S22 Ultra 5G (12GB/2...,Điện thoại Smartphone,Samsung,1,Tiki Trading
5,175845367,Điện Thoại Samsung Galaxy S20 FE (8GB/256GB) -...,Điện thoại Smartphone,Samsung,1,Tiki Trading
6,201703358,Điện thoại Samsung Galaxy A23 5G (4GB/128GB) -...,Điện thoại Smartphone,Samsung,1,Tiki Trading
7,195247535,Điện thoại OPPO Reno8 5G (8GB/256GB) - Hàng ch...,Điện thoại Smartphone,OPPO,213861,OPPO Official Store
8,172826900,Điện thoại Samsung Galaxy A13 (4GB/128GB) - Hà...,Điện thoại Smartphone,Samsung,162514,Long Hưng Mobile
9,216174917,Điện thoại Samsung Galaxy S23 Ultra 5G (8GB/25...,Điện thoại Smartphone,Samsung,1,Tiki Trading


In [ ]:
    if masterProduct is not None:
        if productDetail is not None and Marketing is not None:
            print('------Data is available-------')
    df_MasterProduct=pd.DataFrame(data=masterProduct,columns=['ID','Name','Category','Brand','Shop ID','Shop name'])
    df_ProductDetail=pd.DataFrame(data=productDetail,columns=['Product ID','SKU','Option','Price'])
    df_Marketing=pd.DataFrame(data=Marketing,columns=['Product ID','Count review','Average score','Sold items'])
    
    print('-------Crawl data successfully-------')
    # Dataframe to Excel 
    # df_MasterProduct.to_excel('Master Product.xlsx')
    # df_ProductDetail.to_excel('Product Detail.xlsx')
    # df_Marketing.to_excel('Marketing.xlsx')
    
    #DataFrame to MySQL
    df_MasterProduct.to_sql('master product',con=engine,if_exists='append',index=False)
    df_ProductDetail.to_sql('product detail',con=engine,if_exists='append',index=False)
    df_Marketing.to_sql('marketing',con=engine,if_exists='append',index=False)

In [ ]:
mastre

In [33]:
def Get_Product_Information():
    masterProduct=[]
    productDetail=[]
    Marketing=[]
    for id in product_list:
        response=requests.get(f'https://tiki.vn/api/v2/products/{id}',headers=headers,params=params)
        text=response.content
        data=json.loads(text)

        master_=[]
        master_.append(data['id'])
        master_.append(data['name'])
        master_.append(data['categories']['name'])
        master_.append(data['brand']['name'])
        master_.append(data['current_seller']['id'])
        master_.append(data['current_seller']['name'])
        masterProduct.append(master_)
        
        for i in range(0,len(data['configurable_products'])):
            detail_=[]
            detail_.append(data['id'])
            detail_.append(int(data['configurable_products'][i]['sku']))
            detail_.append(data['configurable_products'][i]['option1'])
            detail_.append(data['configurable_products'][i]['price'])
            productDetail.append(detail_)
            
        marketing_=[]
        marketing_.append(data['id'])
        marketing_.append(data['review_count'])
        marketing_.append(data['rating_average'])
        marketing_.append(data['all_time_quantity_sold'])
        Marketing.append(marketing_)

    if masterProduct is not None:
        if productDetail is not None and Marketing is not None:
            print('------Data is available-------')
    df_MasterProduct=pd.DataFrame(data=masterProduct,columns=['ID','Name','Category','Brand','Shop ID','Shop name'])
    df_ProductDetail=pd.DataFrame(data=productDetail,columns=['Product ID','SKU','Option','Price'])
    df_Marketing=pd.DataFrame(data=Marketing,columns=['Product ID','Count review','Average score','Sold items'])
    
    print('-------Crawl data successfully-------')
    # Dataframe to Excel 
    # df_MasterProduct.to_excel('Master Product.xlsx')
    # df_ProductDetail.to_excel('Product Detail.xlsx')
    # df_Marketing.to_excel('Marketing.xlsx')
    
    #DataFrame to MySQL
    df_MasterProduct.to_sql('master product',con=engine,if_exists='append',index=False)
    df_ProductDetail.to_sql('product detail',con=engine,if_exists='append',index=False)
    df_Marketing.to_sql('marketing',con=engine,if_exists='append',index=False)

